In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

%matplotlib inline

In [ ]:
plt.style.use("jf_cb")

In [ ]:
import numericalunits as nu

nu.Debye = 0.020819434 * nu.e * nu.nm

In [ ]:
import sys

sys.path.append("/share/apps/sphere_greens_function")
from calc_sphere_gf import calc_sphere_gf, DrudeAgDelga

In [ ]:
# sys.path.append('..')
from spectral_density_fit import spectral_density_fitter

# Spectral density

In [ ]:
def Jωprefac(ω, μ):
    "input should be in numericalunits units"
    return ω**2 * μ**2 / (np.pi * nu.hbar * nu.eps0 * nu.c0**2)

In [ ]:
def sphere_Jω(R, ϵsphere, ϵbg, ħω, rsource, μ, nmax=70, **kwargs):
    "input should be in numericalunits units"
    GF = calc_sphere_gf(R / nu.nm, ϵbg, ħω / nu.eV, rsource / nu.nm, rsource / nu.nm, nmax=nmax, epssphere=ϵsphere, **kwargs)
    pref = Jωprefac(ħω / nu.ħ, μ)
    return pref * GF.imag.squeeze() / nu.m

In [ ]:
ωes = np.r_[3.3]  # eV
# use emitter with 15 Debye (quite large)
μes = np.r_[15] * nu.Debye  # numericalunits units

In [ ]:
R = 5.0  # nm
epsbg = 2.1
epssphere = DrudeAgDelga  # Drude model for silver from Alex Delga PRL
rsource = np.array([[0, 0, -R - 1.0]])  # nm
ω = np.linspace(0.5, 7.5, 701)  # eV
Jω = sphere_Jω(R * nu.nm, epssphere, epsbg, ω * nu.eV, rsource * nu.nm, μes, nmax=50)
Jω = nu.hbar * Jω[2, 2, :] / nu.eV  # now in eV, and zz component

plt.plot(ω, Jω)
for ωe in ωes:
    plt.axvline(ωe, ls="--", color="C1")
plt.xlabel("ω (eV)")
plt.ylabel("J(ω) (eV)")
plt.yscale("log")
plt.tight_layout(pad=0.5);

# Fit spectral density to few-mode model

In [ ]:
# try to fit with 4 modes
Nm = 4
# we have 1 emitter
Ne = 1

H = np.diag([3.05, 3.2, 3.4, 3.45])
κ = np.r_[0.1, 0.1, 0.12, 0.13]
g = np.r_[0.03, 0.04, 0.1, 0.1].reshape(1, Nm)  # g has to be Ne x Nm array

# make fitter object (is actually an nlopt object, with some functions added by spectral_density_fit)
opt = spectral_density_fitter(ω, Jω, Nm)
# get the 1d parameter array ps from initial guesses for H,κ,g
ps = opt.Hκg_to_ps(H, κ, g)
# evaluate the model spectral density with those parameters
# .squeeze() because Jfun returns [Ne,Ne,Nω] array also for Ne=1, squeeze removes dimensions of size 1
# (i.e., transforms Jmod.shape from [1,1,len(ω)] -> [len(ω)])
Jmod = opt.Jfun(ω, ps).squeeze()

plt.plot(ω, Jω, label="numerical")
plt.plot(ω, Jmod, label="guessed model")
plt.xlabel("ω (eV)")
plt.ylabel("J(ω) (eV)")
plt.yscale("log")
plt.legend()
plt.tight_layout(pad=0.5);

In [ ]:
ps = opt.optimize(ps)
Jmod = opt.Jfun(ω, ps).squeeze()

plt.plot(ω, Jω, label="numerical")
plt.plot(ω, Jmod, label="fitted model")
plt.xlabel("ω (eV)")
plt.ylabel("J(ω) (eV)")
plt.yscale("log")
plt.legend()
plt.tight_layout(pad=0.5);

# Solve dynamics

In [ ]:
tsfs = np.linspace(0, 50, 201)
# since we have ħ=1 and energy units of eV, time units in mesolve are ħ/eV (~0.658 fs)
# tsfs*nu.fs converts to "internal" numericalunits time unit, dividing by nu.ħ/nu.eV converts that to ħ/eV
ts = tsfs * nu.fs / (nu.ħ / nu.eV)

## With direct discretization

In [ ]:
dω = ω[1] - ω[0]
gdisc = np.sqrt(Jω * dω).reshape(1, -1)
# single-excitation subspace
H_disc = np.block([[np.diag(ω), gdisc.T], [gdisc, np.diag(ωes)]])
ψ0_disc = np.r_[np.zeros_like(ω), 1.0]

In [ ]:
# using sesolve from qutip is really quite inefficient here, but doesn't matter in this case
sol_disc = sesolve(Qobj(H_disc), Qobj(ψ0_disc), ts)

## With few-mode model

We want to treat the system with $N_m$ cavity modes and $N_e$ emitters with Hamiltonian (within RWA)
\begin{equation}
H = \sum_{ij} \omega_{ij} a_i^\dagger a_j + \sum_\alpha \omega_{e,\alpha} \sigma_\alpha^\dagger \sigma_\alpha + \sum_{\alpha,i} g_{\alpha i} (\sigma_\alpha^\dagger a_i + \sigma_\alpha a_i^\dagger)
\end{equation}
and Lindblad decay terms $\kappa_i \mathcal{L}_{a_i}[\rho]$.

The Hamiltonian can be written compactly by defining a vector $\vec{A} = [a_1,a_2,\ldots,a_{N_m},\sigma_1,\ldots,\sigma_{N_e}]^T$ and a block matrix
\begin{equation}
H_{sys} = \begin{pmatrix} \omega & g^T\\g & \mathrm{diag}(\omega_e) \end{pmatrix}
\end{equation}
We can then write $H = \vec{A}^\dagger H_{sys} \vec{A}$. This is exactly how we implement the system below

In [ ]:
# spectral_density_fitter uses jax, convert to normal numpy arrays for further use
ωij, κs, gαi = map(np.array, opt.ps_to_Hκg(ps))
display(Qobj(ωij))
display(Qobj(κs.reshape(1, -1)))
display(Qobj(gαi))

In [ ]:
# make the H_sys matrix
Hsys = np.block([[ωij, gαi.T], [gαi, np.diag(ωes)]])
assert Qobj(Hsys).isherm

In [ ]:
# maximum number of excitations
Nexc = 1
# dimensions of the quantum operators.
# Since the only restriction we want is in the total number of excitations,
# we allow each individual photonic mode to have up to Nexc+1 states (i.e., 0 to Nexc photons),
# while the emitters are two-level systems
part_dims = [Nexc + 1] * Nm + [2] * Ne
print("Dimensions of quantum operators:", part_dims)
print("Full Hilbert space would have size", np.prod(part_dims))

# this creates a list of annihilation operators for subsystems with dimensions given by part_dims,
# but only allowing up to Nexc excitations in the system
ann_ops = enr_destroy(part_dims, Nexc)
# the first Nm operators are the photon mode operators a_i
aops = ann_ops[:Nm]
# the rest are the Ne emitter operators σ_α
σs = ann_ops[Nm:]
assert len(σs) == Ne
print("excitation-number restricted Hilbert space for up to", Nexc, "excitations has size", σs[0].shape[0])

In [ ]:
# Hamiltonian
H = sum(Hsys[i, j] * ann_ops[i].dag() * ann_ops[j] for i in range(len(ann_ops)) for j in range(len(ann_ops)))
# decay terms operators
c_ops = [np.sqrt(ka) * a for (ka, a) in zip(κs, aops)]
# calculate, e.g., expected populations of all subsystems
e_ops = [x.dag() * x for x in ann_ops]

In [ ]:
# start with the emitter excited
ψ0 = enr_fock(part_dims, Nexc, np.r_[np.zeros(Nm), 1])

In [ ]:
sol = mesolve(H, ψ0, ts, c_ops, e_ops=e_ops)

In [ ]:
plt.plot(tsfs, sol.expect[-1], label="emitter")
plt.plot(tsfs, [abs(ψ[-1].item()) ** 2 for ψ in sol_disc.states], "--", label="emitter (direct discretization)")

for ii in range(Nm):
    plt.plot(tsfs, sol.expect[ii], label=f"photon mode {ii + 1}")
plt.xlabel("t (fs)")
plt.ylabel("population")
plt.legend()
plt.tight_layout(pad=0.5);